# 第9章：开发者工具

## CUDA Python性能

为了在CUDA中实现最佳性能,您必须考虑几个因素:
- 本地化内存访问以最小化内存延迟。
- 最大化每个多处理器的活动线程数,以确保硬件的高利用率。
- 最小化条件分支。

为了克服CPU和GPU之间通过PCIe总线的瓶颈,我们希望:
- 最小化传输的数据量。通过大批量传输数据可以最小化数据传输操作的次数。
- 以补充硬件架构的方式组织数据。
- 利用异步传输功能,允许计算和数据传输同时进行。通过计算与数据传输的重叠可以隐藏数据传输造成的延迟。

## 常见陷阱
最常见的错误是在GPU节点上运行仅CPU的代码。只有专门编写为在GPU上运行的代码才能利用GPU。确保您的代码使用正确的GPU加速库、驱动程序和硬件。

**零GPU利用率**
检查确保您的软件支持GPU。只有专门编写为使用GPU的代码才能利用它们。
确保您的软件环境配置正确。在某些情况下,您的代码需要某些库才能在GPU上运行。检查您的依赖项、CUDA工具包版本和软件环境要求。
 
**低GPU利用率** (例如低于~15%)
使用超过必要数量的GPU。您可以通过执行扩展性分析来找到最佳的GPU和CPU核心数量。
检查您的进程的吞吐量。如果您正在向慢速内存写入输出、进行不必要的复制或在CPU和GPU之间切换,您可能会看到低利用率。

**内存错误**
访问违规错误。读取或写入不允许或不允许的内存位置可能导致不可预测的行为和系统崩溃。
内存泄漏。当内存被分配但未正确释放时,应用程序将消耗GPU内存资源,但不会利用它们。分配的内存将不可用于进一步计算。


# CUDA Python开发者工具入门

## 前提条件

本文档中的步骤假设用户有一个能够在GPU上运行CuPy和Numba代码的环境。请参阅这些相应项目进行设置。

- [Nsight Systems](https://developer.nvidia.com/nsight-systems) (也可在[CUDA工具包](https://developer.nvidia.com/cuda-toolkit)中获得)
- [Nsight Compute](https://developer.nvidia.com/nsight-compute) (也可在[CUDA工具包](https://developer.nvidia.com/cuda-toolkit)中获得)
- [nvtx Python绑定](https://pypi.org/project/nvtx/)  




## 使用Nsight Systems进行性能分析

[Nsight Systems](https://developer.nvidia.com/nsight-systems)是一个平台性能分析工具,旨在为用户提供整个平台性能活动的高级、时间相关视图。这包括CPU、GPU、内存、网络、操作系统和应用程序级别的指标。它帮助识别最大的优化机会,并调整以有效地扩展所有可用资源。本教程只会涉及Nsight Systems功能的皮毛。有关完整详细信息,请参阅[文档](https://docs.nvidia.com/nsight-systems/)。

## 使用Nsight Systems GUI设置性能分析

打开Nsight Systems GUI后,选择要进行性能分析的目标机器。这可以是本地机器或远程服务器。此示例使用本地目标。要使用Nsight Systems对Python工作负载进行性能分析,将"Command line with arguments:"字段设置为指向Python解释器和要运行的Python文件(包括任何参数)。确保Python可执行文件在具有应用程序所需的所有依赖项的环境中。例如:"C:\Users\myusername\AppData\Local\miniconda3\python.exe C:\Users\myusername\cupyTests\cupyProfilingStep1.py \<args if needed\>"

还要填写Python可执行文件应该运行的"Working directory"。

**推荐的设置/标志**

对Python进行性能分析的一个好的初始标志集包括:
- 收集CPU上下文切换跟踪
- 收集CUDA跟踪
- 收集GPU指标
- Python性能分析选项:
  - 收集Python回溯样本

您可以在[这里](https://docs.nvidia.com/nsight-systems/UserGuide/index.html#profiling-from-the-gui)了解更多关于所有选项的信息

# CuPy性能分析示例

在此示例中,我们创建两个CuPy数组。然后对其中一个进行排序并计算点积。

In [ ]:
import sys
import cupy as cp


def create_array(x, y) :
    return cp.random.random((x, y),dtype=cp.float32)

def sort_array(a) :
    return cp.sort(a)

def run_program() :
    print("初始化步骤...")
    arr1 = create_array(10_000, 10_000)
    arr2 = create_array(10_000, 10_000)

    print("排序步骤...")
    arr1 = sort_array(arr1)

    print("点积步骤...")
    arr3 = cp.dot(arr1, arr2)
    
    print("完成")
    return

if __name__ == '__main__':

    run_program()


**步骤1 - 对CuPy工作负载进行性能分析**

首先,使用上述设置和标志对这个CuPy示例运行初始性能分析。如果无法从GUI启动性能分析,也可以从命令行启动性能分析。下面是运行此分析的示例CLI命令。根据您的具体设置,某些标志可能会有所不同。

*nsys profile --gpu-metrics-device=all --python-sampling=true --python-sampling-frequency=1000 --trace=cuda --cpuctxsw=process-tree python "/home/myusername/cupytest1.py"*


性能分析完成后,在时间线上的**Processes**行下找到Python进程线程。通过左键单击并拖动感兴趣区域来选择它,放大到Python线程的活动部分。然后右键单击选择"放大到选择"。如果您将鼠标悬停在**Python Backtrace**行中的样本上,将出现一个弹出窗口,显示采样时正在执行的调用堆栈。

![cupy1](images/chapter-09/cupy-profiling-1.png)

CuPy在执行时会在底层调用CUDA内核。Nsight Systems会自动检测这些内核。展开**CUDA HW**行以查看内核的调度位置。

![cupy2](images/chapter-09/cupy-profiling-2.png)

查看**GPU Metrics > GPU Active**和**SM Instructions**行以验证GPU正在被使用。您可以将鼠标悬停在此行上的某个点以查看利用率百分比。

![cupy3](images/chapter-09/cupy-profiling-3.png)

**步骤2 - 添加nvtx**

Nsight Systems可以自动检测CUDA内核以及许多其他框架或库的API。此外,[nvtx](https://github.com/NVIDIA/NVTX)注释模块使用户能够标记自己的应用程序,以在时间线上查看个性化的跟踪事件和范围。[nvtx Python模块](https://pypi.org/project/nvtx/)可通过pip获得,可以使用以下命令安装:

*pip install nvtx*

下面的代码为CuPy应用程序添加了nvtx,在工作负载的各个阶段定义了彩色范围。运行这个新版本的性能分析以查看时间线上的nvtx。如果使用CLI,将标志更新为"--trace=nvtx,cuda"



In [2]:
import sys
import cupy as cp
import nvtx

def create_array(x, y) :
    return cp.random.random((x, y),dtype=cp.float32)

def sort_array(a) :
    return cp.sort(a)

def run_program() :
    print("初始化步骤...")
    nvtx.push_range("init_step", color='green')
    arr1 = create_array(10_000, 10_000)
    arr2 = create_array(10_000, 10_000)
    nvtx.pop_range()

    print("排序步骤...")
    nvtx.push_range("sort_step", color='yellow')
    arr1 = sort_array(arr1)
    nvtx.pop_range()

    nvtx.push_range("dot_step", color='magenta')
    print("点积步骤...")
    arr3 = cp.dot(arr1, arr2)
    nvtx.pop_range()
    
    print("完成")
    return

if __name__ == '__main__':
    
    nvtx.push_range("run_program", color='white')
    run_program()
    nvtx.pop_range()

初始化步骤...
排序步骤...
点积步骤...
完成


Python进程的CPU线程的**NVTX**行显示CPU何时在这些范围内。CUDA HW部分下的**NVTX**行显示这些范围在GPU上何时处于活动状态。注意它们并不完全对齐,这是因为GPU执行调度。您还可以看到CUDA内核如何映射到这些nvtx范围,这些范围代表我们工作负载的各个阶段。

在这个特定示例中,我们可以在**GPU Metrics > SM Instructions > Tensor Active**行中看到,当内核运行时GPU的张量核心并未被激活。张量核心可以为计算密集型内核增加很多性能。下一步将是激活它们。

![cupy4](images/chapter-09/cupy-profiling-4.png)

**步骤3 - 启用TF32**

在Ampere架构上,我们可以通过启用TF32来利用张量核心。这可以通过设置环境变量来完成。

在Linux上:
```
export CUPY_TF32=1
```

在Windows上:
```
set CUPY_TF32=1
```

或者在Python代码中:

```
import os
os.environ["CUPY_TF32"] = "1"
```

让我们在代码中添加这个设置并运行另一个性能分析。



In [ ]:
import sys
import cupy as cp
import nvtx
import os



def create_array(x, y) :
    return cp.random.random((x, y),dtype=cp.float32)

def sort_array(a) :
    return cp.sort(a)

def run_program() :
    print("初始化步骤...")
    nvtx.push_range("init_step", color='green')
    arr1 = create_array(10_000, 10_000)
    arr2 = create_array(10_000, 10_000)
    nvtx.pop_range()

    print("排序步骤...")
    nvtx.push_range("sort_step", color='yellow')
    arr1 = sort_array(arr1)
    nvtx.pop_range()

    nvtx.push_range("dot_step", color='magenta')
    print("点积步骤...")
    arr3 = cp.dot(arr1, arr2)
    nvtx.pop_range()
    
    print("完成")
    return

if __name__ == '__main__':
    os.environ["CUPY_TF32"] = "1"
    nvtx.push_range("run_program", color='white')
    run_program()
    nvtx.pop_range()

![cupy5](images/chapter-09/cupy-profiling-5.png)

**注意**现在在点积期间张量核心正在被使用,GPU上点积范围的运行时间从312ms缩短到116ms。

**步骤4 - 使用注释文件**
Nsight Systems还可以使用注释文件自动跟踪特定Python模块的函数,在本例中是CuPy。此示例指向文件"cupy_annotations.json",其中包含:
```
[
    {
        "_comment": "CuPy注释",
        
        "module": "cupy",
   "color": "black",
        "functions": ["random.random","dot","sort"]
    }

]
```
这个json对象指示应该跟踪来自模块"cupy"的函数"random.random"、"dot"和"sort",并在时间线上显示为黑色范围。如下所示,将此文件添加到配置中的"Python Functions trace"字段。

![cupy6](images/chapter-09/cupy-profiling-6.png)

要从CLI执行此操作,添加类似" --python-functions-trace=\"/home/myusername/cupy_annotations.json\" "的标志
运行另一个性能分析以查看自动跟踪。

![cupy7](images/chapter-09/cupy-profiling-7.png)


# Numba性能分析示例

虽然Nsight Systems显示了平台范围的性能分析信息和一些GPU特定数据(如GPU指标),但它不会深入研究GPU内核本身。这就是[Nsight Compute](https://developer.nvidia.com/nsight-compute)的用武之地。Nsight Compute对在GPU上运行的内核进行详细的性能分析。历史上,这些内核是用本地语言(如C)编写的,但像Numba这样的新技术也使Python开发人员能够编写内核。本节将描述如何使用Nsight Compute对Numba内核进行性能分析。有关Nsight Compute的详细信息,请查看[Nsight Compute文档](https://docs.nvidia.com/nsight-compute/)。


**使用Nsight Compute GUI设置性能分析**

要使用Nsight Compute对Numba应用程序进行性能分析,从GUI打开"Connect"对话框。选择python解释器二进制文件作为"Application Executable"。确保此解释器在具有应用程序所需的所有依赖项的环境中运行,例如支持Numba的Conda shell。然后填写"Working Directory"字段,并在"Command Line Arguments"字段中放入您的Python文件和任何其他命令行参数。这告诉Nsight Compute如何启动您的工作负载进行性能分析。

![numba1](images/chapter-09/numba-profiling-1.png)

**推荐的设置/标志**

Nsight Compute有很多选项来配置您的性能分析。本指南并不打算涵盖所有这些选项,但在[文档](https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html#command-line-options)中有很多额外信息。Numba性能分析的一个好的起点是选择**Profile**活动。在**Filter > Kernel Base Name下拉菜单中选择"Demangled"**。在**Other > Enable CPU Call Stack**中选择Yes,**Other > CPU Call Stack Types**中选择All或Python。

**Metrics**标签是您选择要收集哪些性能指标的地方。指标被分组成集合,详细集合是一个好的起点。您可以在[内核性能分析指南](https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html)中了解更多关于指标的信息。更新这些设置后,单击**Launch**开始自动性能分析过程。Nsight Compute将通过多遍重放机制对遇到的每个内核进行性能分析,并在完成后报告结果。如果无法从GUI进行性能分析,您可以从GUI配置性能分析并从**Common**标签中的"Command Line:"复制适当的命令。此性能分析的示例命令可能是:

*ncu --config-file off --export "\home\myusername\r%i" --force-overwrite --launch-count 3 --set detailed --call-stack --call-stack-type native --call-stack-type python --nvtx --import-source yes \home\myusername\numbaTest1.py*


### Nsight Compute性能分析演练示例

在这个简单的示例中,有一个Numba内核在做向量加法。它接收三个向量,将两个相加,并在第三个向量中返回和。注意"@cuda.jit"装饰器有参数"(lineinfo=True)"。这对于将内核性能数据解析到源代码行很重要。使用上述设置,启动性能分析以查看内核的性能。


In [ ]:
import numpy as np
from numba import cuda


@cuda.jit(lineinfo=True)
def vecadd(a, b, c):
    tid = cuda.grid(1)
    size = len(c)
    if tid < size:
        c[tid] = a[tid] + b[tid]

def run_program() :


    np.random.seed(1)


    N = 500000


    a = cuda.to_device(np.random.random(N))
    b = cuda.to_device(np.random.random(N))
    #a = cuda.to_device(np.float32(np.random.random(N)))
    #b = cuda.to_device(np.float32(np.random.random(N)))
    c = cuda.device_array_like(a)


    vecadd.forall(len(a))(a, b, c)
    print(c.copy_to_host())

When the profile completes, the **Summary** page shows an overview of the kernels profiled. In this example, it’s only one. Expanding the “Demangled Name” column shows that this is the “vecadd” kernel that we wrote with Numba. The Summary has some basic information including the kernel duration and compute and memory throughput. It also lists top performance rules that were triggered and estimated speedups for correcting them. 

![numba2](images/chapter-09/numba-profiling-2.png)

Double clicking on the kernel will open the **Details** page with much more information.

The “GPU Speed of Light Throughput” section at the top shows that this kernel has much higher Memory usage than Compute. The Memory Workload Analysis section shows significant traffic to device memory. 

![numba3](images/chapter-09/numba-profiling-3.png)

The Compute Workload Analysis section shows the majority of the compute is using the FP64 pipeline. 

![numba4](images/chapter-09/numba-profiling-4.png)

The Source Counters section at the bottom shows the source locations with the most stalls and clicking on one opens the **Source** page. 

![numba5](images/chapter-09/numba-profiling-5.png)

Since this was a very simple kernel, most of the stalls are on the addition statement, but with more complex kernels, this level of detail is invaluable. Additionally, the **Context** page will show the CPU call stack that led to this kernel being executed. 

![numba6](images/chapter-09/numba-profiling-6.png)

For this example, we did not specify the data type in Numpy which defaulted to FP64. This caused an increase in memory traffic that was unintended. To manually switch to using the FP32 datatype switch these lines:
    
    a = cuda.to_device(np.random.random(N))
    b = cuda.to_device(np.random.random(N))
    
to this:

    a = cuda.to_device(np.float32(np.random.random(N)))
    b = cuda.to_device(np.float32(np.random.random(N)))

After switching to the FP32 datatype and rerunning a profile, we can see that the runtime of the kernel decreased significantly as did the memory traffic. Setting the initial result to the [Baseline](https://docs.nvidia.com/nsight-compute/NsightCompute/index.html#id7) and opening up the new result will automatically compare the two. Notice that the FP64 usage has disapperared and the kernel has sped up from 59us to 33us. 

![Img7](images/chapter-09/numba-profiling-7.png)


Nsight Compute has an abundance of performance data and built-in expertise. Each section on the Details page has detailed information for a particular category of metrics including Guided Analysis rules and descriptions. The best way to learn about all these features is to try it out on your workload and use the documentation and collateral to assist.
